In [1]:
import datasets
from tqdm.notebook import tqdm
import numpy as np
from substitutions import tenk_word_pairs as word_pairs
from collections import Counter
import json

In [2]:
ds_path = './17e7_tokens_perturbed.hf/'
num_proc = 16
n_per_sub = 1000
seed = 1
just_one = True

In [ ]:
ds = datasets.load_from_disk(ds_path)

In [ ]:
ds

In [ ]:
swap_arr = np.array(ds["substitutions"])
print(swap_arr.shape)

In [ ]:
# This random state allows the perturbations to be reproducible
rs = np.random.RandomState(seed=seed)

#take the sequences to perturb
do_sub = []
indexes = []
for i, (w1, w2) in tqdm(enumerate(word_pairs), total=len(word_pairs)):
    # create indices
    idx = np.arange(len(swap_arr))
    has_sub = idx[swap_arr[:, i] == 1]
    rs.shuffle(has_sub)
    
    all_indexes = has_sub[:n_per_sub]
    labels = rs.randint(0, 2, size=n_per_sub).astype(bool)
    
    # just for checksum
    do_sub.append(all_indexes[labels])
    indexes.append(all_indexes)

In [ ]:
np.sum([ i.sum() for i in do_sub ])

In [ ]:
len(all_indexes)

In [ ]:
if just_one:
    subset_ds = ds.select(all_indexes)
else:
    subset_ds = ds.select(indexes)

In [ ]:
#saves the data
subset_ds.remove_columns(['hash', 'is_original', 'substitutions'])
subset_ds.to_json(f'debugging_dataset.jsonl', num_proc=num_proc)